In [1]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import jupyros as jr
import rospy
from nav_msgs.msg import Odometry
from matplotlib import animation, rc
from geometry_msgs.msg import Twist
import ipywidgets as widgets
from IPython.display import display
from rt2_assignment1.srv import Command
from ipywidgets import Button, Layout, ButtonStyle, GridBox, VBox, HBox
import ipywidgets as widgets
import time
from matplotlib import animation, rc
import actionlib
from rt2_assignment1.msg import go_to_pointActionResult


rospy.init_node('robot_control_node')



In [2]:

x_linear=0.3
z_angular=0.5

a = widgets.FloatSlider(min=0.3, max=1.0, description="linear")
display(a)

b = widgets.FloatSlider(min=0.5, max=1.0, description="angular")
display(b)

velocities_pub = rospy.Publisher('velocities', Twist, queue_size=10)
velocities_msg = Twist()
velocities_msg.linear.x=x_linear
velocities_msg.angular.z=z_angular

def on_value_change(change):
    """Callback for when the slider of the linear velocity is changed, it updates the variable x_linear"""
    global x_linear,velocities_msg
    x_linear= change['new']
    velocities_msg.linear.x = change['new']
    velocities_pub.publish(velocities_msg)
    
    
def on_valueang_change(change): 
    """Callback for when the slider of the angular velocity is changed, it updates the variable z_linear"""
    global z_angular,velocities_msg
    z_angular= change['new']
    velocities_msg.angular.z = change['new']
    velocities_pub.publish(velocities_msg)

a.observe(on_value_change, names='value')
b.observe(on_valueang_change, names='value')

FloatSlider(value=0.3, description='linear', max=1.0, min=0.3)

FloatSlider(value=0.5, description='angular', max=1.0, min=0.5)

In [3]:
ui_client = rospy.ServiceProxy('/user_interface', Command)

button_start = widgets.Button(description="start",layout=Layout(width='auto'))
button_stop = widgets.Button(description="stop when target reached",layout=Layout(width='auto'))
button_stop_now = widgets.Button(description="stop now",layout=Layout(width='auto'))
output = widgets.Output()
display(button_start,button_stop,button_stop_now)

#handle changes, the observe method of the widget can be used to register a callback.
def on_button_start_clicked(b):
    """Callback for when the button start is pressed, it sends a request to the /user_interface service"""
    with output:
        print("Starting the robot.")
        ui_client("start")
def on_button_stop_clicked(b):
    """Callback for when the button stop is pressed, it sends a request to the /user_interface service"""
    with output:
        print("Robot will stop when target reached.")
        ui_client("stop")
def on_button_stop_now_clicked(b):
    """Callback for when the button stop now is pressed, it sends a request to the /user_interface service"""
    with output:
        print("Robot stopped.")    
        ui_client("stop_now")
        
button_start.on_click(on_button_start_clicked)
button_stop.on_click(on_button_stop_clicked)
button_stop_now.on_click(on_button_stop_now_clicked)

Button(description='start', layout=Layout(width='auto'), style=ButtonStyle())

Button(description='stop when target reached', layout=Layout(width='auto'), style=ButtonStyle())

Button(description='stop now', layout=Layout(width='auto'), style=ButtonStyle())

In [4]:

cmd_vel_msg = Twist()
cmd_vel_msg.linear.x=0
cmd_vel_msg.angular.z=0

cmd_vel_pub = rospy.Publisher('cmd_vel', Twist, queue_size=10)

forward = widgets.Button(description="forward",layout=Layout(width='default'),align="center",style=ButtonStyle(button_color='lightgreen'))
turn_right = widgets.Button(description="turn right",layout=Layout(width='auto'),style=ButtonStyle(button_color='moccasin'))
turn_left = widgets.Button(description="turn left",layout=Layout(width='auto'),style=ButtonStyle(button_color='moccasin'))
backward = widgets.Button(description="backward",layout=Layout(width='default'),align="center",style=ButtonStyle(button_color='gold'))
stop = widgets.Button(description="stop",layout=Layout(width='default'),style=ButtonStyle(button_color='salmon'))
output = widgets.Output()
#display(forward,turn_right,turn_left,backward,stop,output)

#handle changes, the observe method of the widget can be used to register a callback.
def on_button_forward_clicked(b):
    """Callback for when the button forward is pressed, it sends publish to the /cmd_vel topic"""
    with output:
        global cmd_vel_msg,x_linear,z_angular,ui_client
        ui_client("stop_now")
        time.sleep(1/20)                  # the ui_client cancel the goal and stops the robot by publishing zero velocities in cmd_vel,
        print("going straight forward.")  # we need to wait some time in order to be sure to publish the new velocities after that
        cmd_vel_msg.linear.x=x_linear
        cmd_vel_msg.angular.z=0
        cmd_vel_pub.publish(cmd_vel_msg)
                           
def on_button_backward_clicked(b):
    """Callback for when the button backward is pressed, it sends publish to the /cmd_vel topic"""
    with output:
        global cmd_vel_msg,x_linear,z_angular,ui_client
        ui_client("stop_now")
        time.sleep(1/20)                    # the ui_client cancel the goal and stops the robot by publishing zero velocities in cmd_vel,
        print("going straight backward.")   # we need to wait some time in order to be sure to publish the new velocities after that
        cmd_vel_msg.linear.x=-x_linear
        cmd_vel_msg.angular.z=0
        cmd_vel_pub.publish(cmd_vel_msg)                           

def on_button_turn_left_clicked(b):
    """Callback for when the button turn left is pressed, it sends publish to the /cmd_vel topic"""
    with output:
        global cmd_vel_msg,x_linear,z_angular,ui_client
        ui_client("stop_now")
        time.sleep(1/20)                   # the ui_client cancel the goal and stops the robot by publishing zero velocities in cmd_vel, 
        print("turning left.")             # we need to wait some time in order to be sure to publish the new velocities after that
        cmd_vel_msg.linear.x=0
        cmd_vel_msg.angular.z=-z_angular
        cmd_vel_pub.publish(cmd_vel_msg) 
                           
def on_button_turn_right_clicked(b):
    """Callback for when the button turn right is pressed, it sends publish to the /cmd_vel topic"""
    with output:
        global cmd_vel_msg,x_linear,z_angular,ui_client
        ui_client("stop_now")
        time.sleep(1/20)                   # the ui_client cancel the goal and stops the robot by publishing zero velocities in cmd_vel, 
        print("turning right.")            # we need to wait some time in order to be sure to publish the new velocities after that
        cmd_vel_msg.linear.x=0
        cmd_vel_msg.angular.z=z_angular
        cmd_vel_pub.publish(cmd_vel_msg)                          
        
def on_button_stop_clicked(b):
    """Callback for when the button stop is pressed, it sends publish to the /cmd_vel topic"""
    with output:
        global cmd_vel_msg,x_linear,z_angular,ui_client
        ui_client("stop_now")
        time.sleep(1/20)                  # the ui_client cancel the goal and stops the robot by publishing zero velocities in cmd_vel 
        print("robot stopped.")           # we need to wait some time in order to be sure to publish the new velocities after that
        cmd_vel_msg.linear.x=0
        cmd_vel_msg.angular.z=0
        cmd_vel_pub.publish(cmd_vel_msg)
        
forward.on_click(on_button_forward_clicked)
backward.on_click(on_button_backward_clicked)
turn_left.on_click(on_button_turn_left_clicked)
turn_right.on_click(on_button_turn_right_clicked)  
stop.on_click(on_button_stop_clicked)  

VBox([forward,HBox([turn_left,turn_right]),backward,stop])

In [5]:
fig, ax = plt.subplots()
x_lin=0
z_ang=0
x_lin_cmd_vel, = ax.plot([], [], 'ro',label='x_lin cmd_vel',linestyle = '-',markersize=1,marker = ',')
z_ang_cmd_vel, = ax.plot([], [], 'ro',label='z_ang cmd_vel',color = 'blue',linestyle = '-',markersize=1,marker = ',')
plt.axis([0, 10, -2, 2])
tick=time.time()
x_data=[]
x_lin_cmd_vel_data=[]
z_ang_cmd_vel_data=[]
ax.set_xlabel('time')  # Add an x-label to the axes.
ax.set_title("velocities")  # Add a title to the axes.
ax.legend()  # Add a legend.

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
def init():
    x_lin_cmd_vel.set_data([], [])
    z_ang_cmd_vel.set_data([], [])
    return (x_lin_cmd_vel,z_ang_cmd_vel)


def cmd_callback(msg):
    global x_lin,z_ang
    x_lin=msg.linear.x
    z_ang=msg.angular.z
    

def animate(i):
    global tick
    x_lin_cmd_vel_data.append(x_lin)
    z_ang_cmd_vel_data.append(z_ang)
    x_data.append(time.time()-tick)
    x_lin_cmd_vel.set_data(x_data, x_lin_cmd_vel_data)
    z_ang_cmd_vel.set_data(x_data, z_ang_cmd_vel_data)
    
    if x_data[len(x_data)-1]>9 :
        plt.axis([x_data[len(x_data)-1]-9, x_data[len(x_data)-1]+1, -2, 2])   
        x_lin_cmd_vel_data.pop(0)
        z_ang_cmd_vel_data.pop(0)
        x_data.pop(0)
        
    return (x_lin_cmd_vel,z_ang_cmd_vel)

jr.subscribe('/cmd_vel', Twist, cmd_callback)
#rospy.Subscriber('/cmd_vel', Twist, cmd_callback)

In [7]:
# call the animator. blit=True means only re-draw the parts that have changed.

anim = animation.FuncAnimation(fig, animate, init_func=init, blit=True,interval=20)

In [8]:
N_succ=0
N_preempt=0

def result_callback(msg):
    global N_succ,N_preempt
    
    if msg.status.status==2:
        N_preempt=N_preempt+1
        
    if msg.status.status==3:
        N_succ=N_succ+1     
    
jr.subscribe('/go_to_point/result', go_to_pointActionResult, result_callback)
#rospy.Subscriber('/go_to_point/result', Twist, cmd_callback)


In [9]:
print(N_succ)
#FAI BOXPLOT!!

0
